## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Compute the camera calibration using chessboard images

In [ ]:
# Create an undistort algorithm
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
%matplotlib qt5

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# use cal_undistort, from above

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        
cv2.destroyAllWindows()
%matplotlib inline

for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original ' + fname.split('/')[-1], fontsize=40)
    ax2.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
    ax2.set_title('Undistorted ' + fname.split('/')[-1], fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
pickle.dump([objpoints, imgpoints], open( "objimg_points.p", "wb" ) )


## Create pipeline

### 1.  Undistort image

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
%matplotlib inline

oipoints = pickle.load( open( "objimg_points.p", "rb" ) )
objpoints=oipoints[0]
imgpoints=oipoints[1]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (1280, 720), None, None)

fname = '../test_images/test1.jpg'
img = cv2.imread(fname)
undistorted = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('../report/test1_undist.jpg',undistorted) 

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.set_title('Original ' + fname.split('/')[-1], fontsize=40)
ax2.set_title('Undistorted ' + fname.split('/')[-1], fontsize=40)
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax2.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))



### 2. Apply sobel gradient transform

In [ ]:
def magx_thresh(image, sobel_kernel=3, mag_thresh=35):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    abs_sobelx = np.sqrt(sobelx*sobelx)
    abs_sobely = np.sqrt(sobely*sobely)
    
    abs_sobelxy = np.sqrt(sobelx*sobelx + sobely*sobely)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled_sobelx)
    mag_binary[scaled_sobelx >= mag_thresh] = 1
    # 6) Return this mask as your binary_output image
    return mag_binary, scaled_sobelx

In [ ]:
images = glob.glob('../test_images/*.jpg')
plt.rcParams['figure.figsize'] = (12.8, 7.2)
for fname in images:
    img = cv2.imread(fname)
    fsp = fname.split('/')
    plt.imshow(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    plt.show()
    undistorted = cal_undistort(img, objpoints, imgpoints)

    magb, absx = magx_thresh(undistorted)

    plt.imshow(absx, cmap = 'nipy_spectral', clim = [10, 50])
    plt.title('magx_grad'+' '+fsp[-1][:-4])
    plt.colorbar()
    plt.show()
    
    plt.imshow(magb, cmap = 'gray')
    plt.title('magx_binary'+' '+fsp[-1][:-4])
    plt.colorbar()
    plt.show()

In [ ]:
images = glob.glob('../test_images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    fsp = fname.split('/')
    undistorted = cal_undistort(img, objpoints, imgpoints)

    magxb, absx = magx_thresh(undistorted)

    fsp = fname.split('/')
    newfname = fsp[0]+'/report/'+fsp[2][:-4]+'_thresholded_grad.png'
    #plt.imsave(newfname, grad_thresh, cmap = 'gray')
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('magx_grad ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    ax2.imshow(absx, cmap = 'nipy_spectral', clim = (0, 50))
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('magx_grad ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    ax2.imshow(magxb, cmap = 'gray')

### 3.  Apply Color Transform

In [ ]:
def s_threshold(image, sobel_kernel = 1, thresh=(200, 255)):
    # Calculate s threshold
    
    # Apply the following steps to img
    # 1) Convert to s
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    S = hls[:,:,2]
    # 2) Convert to binary
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary, S

In [ ]:
images = glob.glob('../test_images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    s_thresh, s = s_threshold(undistorted)
    fsp = fname.split('/')
    
    plt.imshow(s, cmap = 'nipy_spectral', clim = (50, 200))
    plt.title('S '+' '+fsp[-1][:-4])
    plt.colorbar()
    plt.show()
    
    plt.imshow(s_thresh*255, cmap = 'gray', clim = (50, 200))
    plt.title('S_binary '+' '+fsp[-1][:-4])
    plt.colorbar()
    plt.show()
    

In [ ]:
images = glob.glob('../test_images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    s_thresh, s = s_threshold(undistorted)
    fsp = fname.split('/')
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('S  ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax2.imshow(s, cmap = 'nipy_spectral')

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('Thresholded s ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), clim = [50,200])
    ax2.imshow(s_thresh, cmap = 'gray')

### 4.  Combine Gradient and  Color Transforms

In [ ]:
images = glob.glob('../test_images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    magxb, absx = magx_thresh(undistorted)
    s_thresh, s = s_threshold(undistorted)

    color_comb = np.dstack(( np.zeros_like(magxb), magxb*255, s_thresh*255))

    combined_binary = np.zeros_like(magxb)
    combined_binary[(magxb == 1) | (s_thresh == 1)] = 1

    fsp = fname.split('/')
    newfname = fsp[0]+'/report/'+fsp[2][:-4]+'_combined_color.png'
    plt.imsave(newfname, color_comb)

    newfname = fsp[0]+'/report/'+fsp[2][:-4]+'_combined_threshold.png'
    plt.imsave(newfname, combined_binary, cmap = 'gray')

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('Thresholded combined color ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    ax2.imshow(color_comb)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original ' + fname.split('/')[-1][:-4], fontsize=40)
    ax2.set_title('Thresholded combined binary ' + fname.split('/')[-1][:-4], fontsize=40)
    ax1.imshow(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    ax2.imshow(combined_binary, cmap = 'gray')


### 5. Perform perspective transform

In [ ]:
def warper(img,src,dst):
    img_size=(img.shape[1],img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped

In [ ]:
images = ['../test_images/straight_lines1.jpg', '../test_images/straight_lines2.jpg']
img = cv2.imread(images[0])
img_size=(img.shape[1],img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
fractx0=.15
fractx1=.435
fracty = 0.66
src = np.float32(
    [[img_size[0] * fractx1,       img_size[1] * fracty],
     [ img_size[0] * fractx0,       img_size[1]         ],
     [ img_size[0] * (1 - fractx0), img_size[1]         ],
     [ img_size[0] * (1 - fractx1), img_size[1] * fracty]])
dst = np.float32(
    [[(img_size[0] /4), 0],
     [(img_size[0] / 4), img_size[1]],
     [(img_size[0] * 3 / 4), img_size[1]],
     [(img_size[0] * 3 / 4), 0]])

print('src =')
print(np.int32([src]))
print('')
print('dst = ')
print(np.int32([dst]))


for fname in images:
    img = cv2.imread(fname)
    img_size=(img.shape[1],img.shape[0])
    
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    und = cv2.polylines(undistorted,np.int32([src]),True,(0,0,255))
    warped = warper(und, src, dst)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(cv2.cvtColor(und, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original ' + fname.split('/')[-1], fontsize=30)
    ax2.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
    ax2.set_title('Warped ' + fname.split('/')[-1], fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
images = glob.glob('../test_images/*.jpg')

for fname in images:
    %matplotlib inline
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    
    magxb, absx = magx_thresh(undistorted)
    s_thresh, s = s_threshold(undistorted)
    color_comb = np.dstack(( np.zeros_like(magxb), magxb*255, s_thresh*255))
    combined_binary = np.zeros_like(magxb)
    combined_binary[(magxb == 1) | (s_thresh == 1)] = 1

    warped = warper(combined_binary, src, dst)
    fsp = fname.split('/')
    
    tn = fsp[2][:-4]
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(tn + ' Raw')
    plt.show()
    plt.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
    plt.title(tn + ' Undistorted')
    plt.show()
    plt.imshow(absx, cmap = 'gray')
    plt.title(tn + ' Gradient Threshold')
    plt.show()
    plt.imshow(s_thresh, cmap = 'gray')
    plt.title(tn + ' Saturation Threshold')
    plt.show()
    plt.imshow(color_comb)
    plt.title(tn + ' Combined Gradient and Saturation Threshold')
    plt.show()
    plt.imshow(warped, cmap = 'gray')
    plt.title(tn + ' Warped Combined Threshold')
    plt.show()
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(cv2.cvtColor(und, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original ' + fname.split('/')[-1], fontsize=30)
    ax2.imshow(warped, cmap = 'gray')
    ax2.set_title('Transformed ' + fname.split('/')[-1], fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

## Create simplified pipeline.  Can start here to test video performance.

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
%matplotlib inline

# load distortion points
oipoints = pickle.load( open( "objimg_points.p", "rb" ) )
objpoints=oipoints[0]
imgpoints=oipoints[1]

img_size=(1280, 720)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

fractx0 = 0.15
fractx1=.435
fracty = 0.66
src = np.float32(
    [[img_size[0] * fractx1,       img_size[1] * fracty],
     [ img_size[0] * fractx0,       img_size[1]         ],
     [ img_size[0] * (1 - fractx0), img_size[1]         ],
     [ img_size[0] * (1 - fractx1), img_size[1] * fracty]])
dst = np.float32(
    [[(img_size[0] /4), 0],
     [(img_size[0] / 4), img_size[1]],
     [(img_size[0] * 3 / 4), img_size[1]],
     [(img_size[0] * 3 / 4), 0]])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

san_result = False

def magx_thresh(image, sobel_kernel=3, mag_thresh=35):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    abs_sobelx = np.sqrt(sobelx*sobelx)
    abs_sobely = np.sqrt(sobely*sobely)
    
    abs_sobelxy = np.sqrt(sobelx*sobelx + sobely*sobely)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled_sobelx)
    mag_binary[scaled_sobelx >= mag_thresh] = 1
    # 6) Return this mask as your binary_output image
    return mag_binary

def s_threshold(image, sobel_kernel=1, thresh=(200, 255)):
    # Calculate s threshold
    
    # Apply the following steps to img
    # 1) Convert to s
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    S = hls[:,:,2]
    # 2) Convert to binary
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary


def transform_image(img):
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    magxb = magx_thresh(undistorted)
    s_thresh = s_threshold(undistorted)
    combined_binary = np.zeros_like(magxb)
    combined_binary[(magxb == 1) | (s_thresh == 1)] = 1
    warped = cv2.warpPerspective(combined_binary, M, img_size)
    return(warped)

### Test pipeline

In [ ]:
images = glob.glob('../test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    warped = transform_image(img)
    fsp = fname.split('/')
    
    tn = fsp[2][:-4]
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original ' + fname.split('/')[-1], fontsize=30)
    ax2.imshow(warped, cmap = 'gray')
    ax2.set_title('Transformed ' + fname.split('/')[-1], fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

### Window from lecture 33

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

images = glob.glob('../test_images/*.jpg')

for fname in images:
    
    img = cv2.imread(fname)
    binary_warped = transform_image(img)

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    y_eval = np.max(ploty)
   
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # find offset
    lx = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    rx = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]
    center_x = (lx + rx)/2.0
    offset_pixel = center_x - 1279/2.0
    offset = offset_pixel * xm_per_pix
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Combine the result with the original image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    ax1.set_title('Lane ' + fname.split('/')[-1], fontsize=30)
        
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    ax2.imshow(out_img)
    ax2.plot(left_fitx, ploty, color='yellow')
    ax2.plot(right_fitx, ploty, color='yellow')
    #ax2.xlim(0, 1280)
    #ax2.ylim(720, 0)
    ax2.set_title('Window ' + fname.split('/')[-1], fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    

    # Now our radius of curvature is in meters
    print('Left  lane curve radius: {l:.0f} m'.format(l=left_curverad)) 
    print('Right lane curve radius: {r:.0f} m'.format(r=right_curverad))
    print('Offset: {o:1.3f} m'.format(o=offset))

## Implement video

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #yaw 
        self.yaw = None


In [ ]:
def get_new_points(binary_warped):

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 10
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    return leftx, lefty, rightx, righty


def get_nxt_points(binary_warped, leftx, rightx):
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    ploty = np.linspace(0, 719, 720)
    left_fit = np.polyfit(ploty, leftx, 2)
    right_fit = np.polyfit(ploty, rightx, 2)

    # Set the width of the windows +/- margin
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & 
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin))
                       & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

 
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty


def process_image(img):

    binary_warped = transform_image(img)
    
    if not (left.detected & right.detected):
        leftx, lefty, rightx, righty = get_new_points(binary_warped)  
    else:
        leftx, lefty, rightx, righty = get_nxt_points(binary_warped, left.bestx, right.bestx)  

    if (len(lefty)<10 | len(leftx)<10):
        left.detected = False
        left_fit = left.best_fit
    else:
        left_fit = np.polyfit(lefty, leftx, 2)
        
    if (len(righty)<10 | len(rightx)<10):
        right.detected = False
        right_fit = right.best_fit
    else:
        right_fit = np.polyfit(righty, rightx, 2)
        
    ploty = np.linspace(0, 719, 720)
            
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    y_eval = np.max(ploty)
   
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature 
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / (2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) /(2*right_fit_cr[0])
    
    curverad = 2/(1/left_curverad + 1/right_curverad)
    
    left_angle_top  = np.arctan(left_fit_cr[1])  * 180 
    right_angle_top = np.arctan(right_fit_cr[1]) * 180
    
    left_angle  = np.arctan(2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])  * 180 
    right_angle = np.arctan(2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1]) * 180
    
    
    # find offset
    lx = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    rx = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]
    center_x = (lx + rx)/2.0
    offset_pixel = center_x - 1279/2.0
    offset = offset_pixel * xm_per_pix
    lane_width = (rx - lx) * xm_per_pix
    
    san_result = True
    if np.absolute(left_curverad)<10:
        san_result = False
    if np.absolute(right_curverad)<10:
        san_result = False
    if np.absolute(offset)>1.5:
        san_result = False
    if lane_width>4:
        san_result = False
    if lane_width<2:
        san_result = False
    if np.abs(left_angle - right_angle)>5:
        san_result = False
    if np.abs(left_angle_top - right_angle_top)>5:
        san_result = False
    if len(leftx)<200:
        san_result = False
    if len(rightx)<200:
        san_result = False
    
    left.detected = san_result
    right.detected = san_result
    
    alpha = 0.05
    if san_result:
        if left.bestx is not None:
            left.bestx = alpha * left_fitx + (1 - alpha) * left.bestx
            left.line_base_pos = left.bestx[719]
            dx = (left.bestx[709] - left.bestx[719]) * xm_per_pix
            dy = 10 * ym_per_pix
            left.yaw = np.arctan(dx/dy) * 180
        else:
            left.bestx = left_fitx
            left.line_base_pos = left.bestx[719]
            dx = (left.bestx[709] - left.bestx[719]) * xm_per_pix
            dy = 10 * ym_per_pix
            left.yaw = np.arctan(dx/dy) * 180
        if right.bestx is not None:
            right.bestx = alpha * right_fitx + (1 - alpha) * right.bestx  
            right.line_base_pos = right.bestx[719]
            dx = (right.bestx[709] - right.bestx[719]) * xm_per_pix
            dy = 10 * ym_per_pix
            right.yaw = np.arctan(dx/dy) * 180
        else:
            right.bestx = right_fitx
            right.line_base_pos = right.bestx[719]
            dx = (right.bestx[709] - right.bestx[719]) * xm_per_pix
            dy = 10 * ym_per_pix
            right.yaw = np.arctan(dx/dy) * 180
        if left.best_fit is not None:
            left.best_fit = alpha * left_fit + (1 - alpha) * left.best_fit
        else:
            left.best_fit = left_fit
        if right.best_fit is not None:
            right.best_fit = alpha * right_fit + (1 - alpha) * right.best_fit
        else:
            right.best_fit = right_fit
        if left.radius_of_curvature is not None:
            left.radius_of_curvature = 1 / (alpha / left_curverad + (1-alpha)/left.radius_of_curvature)
        else:
            left.radius_of_curvature = left_curverad
        if right.radius_of_curvature is not None:
            right.radius_of_curvature = 1 / (alpha / right_curverad + (1-alpha)/right.radius_of_curvature)
        else:
            right.radius_of_curvature = right_curverad
        curverad = 2/(1/left.radius_of_curvature + 1/right.radius_of_curvature)
        

    if left.line_base_pos is not None:
        lx = left.line_base_pos
    if right.line_base_pos is not None:
        rx = right.line_base_pos
    center_x = (lx + rx)/2.0
    offset_pixel = center_x - 1279/2.0
    offset = offset_pixel * xm_per_pix
    lane_width = (rx - lx) * xm_per_pix
    
    yaw = (left.yaw + right.yaw)/2.0

    curvature = 500/left.radius_of_curvature + 500/right.radius_of_curvature
            
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left.bestx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right.bestx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Combine the result with the original image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
        
    font = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(result,'Curvature: {c:.2f} 1/km'.format(c=curvature),
                (10,40), font, 1,(255,255,255),2) 
    
    cv2.putText(result,'Angle: {y:.2f} degrees'.format(y=yaw),
                (10,80), font, 1,(255,255,255),2)    
    
    cv2.putText(result,'Offset: {o:1.2f} m'.format(o=offset),
                (10,120), font, 1,(255,255,255),2) 
    
    cv2.putText(result,'Lane width: {w:1.2f} m'.format(w=lane_width),
                (10,160), font, 1,(255,255,255),2) 
    
    return result

In [ ]:
images = glob.glob('../test_images/*.jpg')
plt.rcParams['figure.figsize'] = (12.8, 7.2)
left = Line()
right = Line()

for fname in images:
    left.detected = False
    left.bestx = None
    left.best_fit = None
    left.radius_of_curvature = None
    left.yaw = None
    right.detected = False
    right.bestx = None
    right.best_fit = None
    right.radius_of_curvature = None
    right.yaw = None
    img = cv2.imread(fname)
    res = process_image(img)
    print(fname)
    plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_output_file = '../output_images/project_video_output.mp4'
clip1 = VideoFileClip("../project_video.mp4", audio=False)
project_clip = clip1.fl_image(process_image)
%time project_clip.write_videofile(project_output_file, audio=False)
